In [1]:
%pip -q install docling quackling llama-index llama-index-llms-openllm
%pip -q install semantic-router semantic-chunkers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
types-requests 2.32.0.20240907 requires urllib3>=2, but you have urllib3 1.26.20 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deepsearch-toolkit 1.0.0 requires urllib3<2.0.0,>=1.26.8, but you have urllib3 2.2.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from docling.document_converter import DocumentConverter
from docling.datamodel.base_models import PipelineOptions
from llama_index.llms.openllm import OpenLLM
from semantic_router.encoders.fastembed import FastEmbedEncoder
from semantic_chunkers import StatisticalChunker

import logging
import os
from dotenv import load_dotenv
from __future__ import annotations
from typing import Annotated, List, Optional
from pydantic import BaseModel, Field, constr
from pydantic_core import from_json
from pydantic import ValidationError

_log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
load_dotenv()

/home/noelo/.pyenv/versions/3.10.14/envs/instvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
source = "/home/noelo/dev/instruct-injest/data/CELEX_32021R1173_EN_TXT.pdf"
converter = DocumentConverter(pipeline_options=PipelineOptions())
result = converter.convert_single(source)
_log.info(len(result.pages))
raw_text = result.output.export_to_markdown()

Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 27750.59it/s]
/home/noelo/.pyenv/versions/3.10.14/envs/instvenv/lib/python3.10/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any iss

In [4]:
class QuestionAndAnswer(BaseModel):
    question: str
    answer: str

class SeedExample(BaseModel):
    context: str
    questions_and_answers: List[QuestionAndAnswer] = Field(None, min_items=3, set=True)

class QNAModel(BaseModel):
    created_by: Annotated[str, Field(None)]
    domain: Annotated[str, Field(None)]
    seed_examples: Annotated[List[SeedExample], Field(None, min_items=5, set=True)]

In [5]:
encoder = FastEmbedEncoder()
chunker = StatisticalChunker(encoder=encoder,enable_statistics=True,plot_chunks=False,min_split_tokens=300, max_split_tokens=2000)
chunks = chunker(docs=[raw_text])

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 13107.20it/s]
2024-09-11 14:18:41 INFO semantic_chunkers.utils.logger Single document exceeds the maximum token limit of 2000. Splitting to sentences before semantically merging.
  5%|▌         | 1/19 [00:00<00:17,  1.06it/s]

Chunking Statistics:
  - Total Documents: 64
  - Total Chunks: 6
  - Chunks by Threshold: 5
  - Chunks by Max Chunk Size: 0
  - Last Chunk: 1
  - Minimum Token Size of Chunk: 210
  - Maximum Token Size of Chunk: 653
  - Similarity Chunk Ratio: 0.83


 11%|█         | 2/19 [00:02<00:17,  1.01s/it]

Chunking Statistics:
  - Total Documents: 69
  - Total Chunks: 7
  - Chunks by Threshold: 6
  - Chunks by Max Chunk Size: 0
  - Last Chunk: 1
  - Minimum Token Size of Chunk: 120
  - Maximum Token Size of Chunk: 686
  - Similarity Chunk Ratio: 0.86


 16%|█▌        | 3/19 [00:03<00:16,  1.01s/it]

...

Chunking Statistics:
  - Total Documents: 71
  - Total Chunks: 4
  - Chunks by Threshold: 3
  - Chunks by Max Chunk Size: 0
  - Last Chunk: 1
  - Minimum Token Size of Chunk: 161
  - Maximum Token Size of Chunk: 453
  - Similarity Chunk Ratio: 0.75


 95%|█████████▍| 18/19 [00:21<00:01,  1.19s/it]

Chunking Statistics:
  - Total Documents: 69
  - Total Chunks: 4
  - Chunks by Threshold: 3
  - Chunks by Max Chunk Size: 0
  - Last Chunk: 1
  - Minimum Token Size of Chunk: 238
  - Maximum Token Size of Chunk: 664
  - Similarity Chunk Ratio: 0.75


100%|██████████| 19/19 [00:22<00:00,  1.16s/it]

Chunking Statistics:
  - Total Documents: 74
  - Total Chunks: 3
  - Chunks by Threshold: 2
  - Chunks by Max Chunk Size: 0
  - Last Chunk: 1
  - Minimum Token Size of Chunk: 203
  - Maximum Token Size of Chunk: 583
  - Similarity Chunk Ratio: 0.67


In [19]:
_log.setLevel(level=logging.INFO)

llm_base = OpenLLM(
    model=os.getenv("MODEL_NAME"), 
    api_base=os.getenv("LLM_URL"),
    api_key=os.getenv("API_KEY") )

In [ ]:
gen_prompt=f"You are a helpful question and answer writing assistant. Given the following context generate 5 seed examples containing 3 question and answer pairs. Ensure that the questions can be answered by the context given. Do not number the pairs. Also generate a description of the contexts. All output MUST be in valid JSON format.\n\nContext:"

json_prompt=f"\n\nHere's a JSON schema to follow: {QNAModel.model_json_schema()}.\n\nOutput a valid JSON object but do not repeat the schema."

for ch in chunks[0]:
    llm_msg = gen_prompt+ch.content+json_prompt
    _log.debug(llm_msg)

    it = llm_base.complete(llm_msg,max_tokens=4000,timeout=120.0)
    
    # Ensure that we just take the json output, sometimes we get some rubbish upfront
    json_start = it.text.find('{')
    extracted_json = it.text[json_start:]

    qna_list=[]
    try:
        res = QNAModel.model_validate(from_json(extracted_json,allow_partial=True,cache_strings='keys'))
        print(type(res),id(res))
        qna_list = qna_list + [res]
        print(len(qna_list))
    except (ValidationError ,ValueError) as e:
        _log.error(e,extracted_json)